In [5]:
pip install PyQt5


   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---- ----------------------------------- 0.8/6.9 MB 6.7 MB/s eta 0:00:01
   --------- ------------------------------ 1.6/6.9 MB 5.2 MB/s eta 0:00:02
   --------------- ------------------------ 2.6/6.9 MB 4.7 MB/s eta 0:00:01
   ---------------- ----------------------- 2.9/6.9 MB 3.8 MB/s eta 0:00:02
   ------------------------ --------------- 4.2/6.9 MB 4.1 MB/s eta 0:00:01
   ----------------------------- ---------- 5.0/6.9 MB 4.1 MB/s eta 0:00:01
   --------------------------------- ------ 5.8/6.9 MB 4.1 MB/s eta 0:00:01
   -------------------------------------- - 6.6/6.9 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 6.9/6.9 MB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.1 MB ? eta -:--:--
    --------------------------------------- 1.0/50.1 MB 5.6 MB/s eta 0:00:09
   - -------------------------------------- 1.8/50.1 MB 4.4 MB/s eta 0:00:12
   -- -----------------

In [ ]:
 pip install pyttsx3

In [3]:
# CareCall: Smart Assistant (Enhanced Theme, Vibrant Design, AI Chat)

from PyQt5 import QtWidgets, QtGui, QtCore
import sys
import datetime
import json
import os
import random
import webbrowser

REMINDERS_FILE = "reminders.json"
CONTACTS_FILE = "contacts.json"
NOTES_FILE = "notes.json"
PROFILE_IMAGE = "profile.png"
BACKGROUND_IMAGE_LIGHT = "background_light.jpg"
BACKGROUND_IMAGE_DARK = "background_dark.jpg"

# Helper Functions
def load_data(filename):
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            return json.load(f)
    return []

def save_data(filename, data):
    with open(filename, 'w') as f:
        json.dump(data, f)

def get_today_reminders(reminders):
    today = datetime.date.today().isoformat()
    return [r for r in reminders if r['date'] == today]

class SidebarButton(QtWidgets.QPushButton):
    def __init__(self, label, icon_path, parent=None):
        super().__init__(label, parent)
        self.setIcon(QtGui.QIcon(icon_path))
        self.setMinimumHeight(48)
        self.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
        self.setStyleSheet("""
            QPushButton {
                text-align: left;
                padding: 10px 16px;
                font-size: 16px;
                font-weight: bold;
                color: #f8fafc;
                background-color: rgba(255,255,255,0.04);
                border: 1px solid transparent;
                border-radius: 10px;
                font-family: 'Segoe UI', sans-serif;
                transition: all 0.3s ease-in-out;
            }
            QPushButton:hover {
                background-color: qlineargradient(x1:0, y1:0, x2:1, y2:1, stop:0 #6366f1, stop:1 #3b82f6);
                color: #ffffff;
                border: 1px solid #4f46e5;
            }
        """)

class AIChatDialog(QtWidgets.QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("CareCall AI Chat")
        self.resize(600, 500)
        self.setStyleSheet("""
            QDialog {
                background-color: #1e293b;
                color: #f1f5f9;
                font-family: 'Segoe UI';
            }
            QTextBrowser {
                background-color: #0f172a;
                padding: 12px;
                border: 1px solid #334155;
                border-radius: 10px;
            }
            QLineEdit {
                padding: 10px;
                border: 1px solid #334155;
                border-radius: 8px;
                background-color: #0f172a;
                color: #e2e8f0;
            }
            QPushButton {
                background-color: #3b82f6;
                color: white;
                padding: 10px 16px;
                border-radius: 8px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #2563eb;
            }
        """)

        layout = QtWidgets.QVBoxLayout(self)

        self.chat_history = QtWidgets.QTextBrowser()
        self.input_field = QtWidgets.QLineEdit()
        self.send_button = QtWidgets.QPushButton("Send")

        form_layout = QtWidgets.QHBoxLayout()
        form_layout.addWidget(self.input_field)
        form_layout.addWidget(self.send_button)

        layout.addWidget(self.chat_history)
        layout.addLayout(form_layout)

        self.send_button.clicked.connect(self.respond)
        self.input_field.returnPressed.connect(self.respond)

    def respond(self):
        user_text = self.input_field.text().strip()
        if not user_text:
            return
        self.chat_history.append(f"<p style='color:#cbd5e1'><b>You:</b> {user_text}</p>")
        response = self.get_response(user_text)
        self.chat_history.append(f"<p style='color:#7dd3fc'><b>CareCall AI:</b> {response}</p>")
        self.input_field.clear()

    def get_response(self, user_input):
        user_input = user_input.lower()
        if "medicine" in user_input:
            return "Don't forget to take your medicine on time."
        elif "hello" in user_input or "hi" in user_input:
            return "Hello there! How can I support you today?"
        elif "how are you" in user_input:
            return "I'm here and ready to help you feel better!"
        elif "sad" in user_input or "depressed" in user_input:
            return "I'm sorry to hear that. Try talking to someone you trust, or take a walk outside."
        elif "thank you" in user_input:
            return "You're most welcome. I'm always here for you!"
        elif "remind me" in user_input:
            return "You can use the Add Reminder option from the sidebar."
        elif "bye" in user_input:
            return "Goodbye! Take care."
        else:
            return "I'm still learning. Try asking about your health, feelings, or reminders."

class CareCallApp(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("CareCall AI Assistant")
        self.setGeometry(0, 0, 1400, 800)
        self.setWindowState(QtCore.Qt.WindowMaximized)

        self.reminders = load_data(REMINDERS_FILE)
        self.contacts = load_data(CONTACTS_FILE)
        self.notes = load_data(NOTES_FILE)
        self.init_demo_data()

        main_widget = QtWidgets.QWidget()
        main_layout = QtWidgets.QHBoxLayout(main_widget)
        self.setCentralWidget(main_widget)

        # Theme & Background
        hour = datetime.datetime.now().hour
        if hour < 18:
            bg = BACKGROUND_IMAGE_LIGHT
            text_color = "#1e293b"
        else:
            bg = BACKGROUND_IMAGE_DARK
            text_color = "#f8fafc"

        if os.path.exists(bg):
            main_widget.setStyleSheet(f"QWidget {{ background-image: url({bg}); background-position: center; background-repeat: no-repeat; background-size: cover; }}")
        else:
            main_widget.setStyleSheet(f"background-color: #0f172a;")

        self.sidebar = QtWidgets.QFrame()
        self.sidebar.setFixedWidth(270)
        self.sidebar.setStyleSheet("background-color: rgba(0,0,0,0.5);")
        sidebar_layout = QtWidgets.QVBoxLayout(self.sidebar)

        profile = QtWidgets.QLabel()
        profile.setPixmap(QtGui.QPixmap(PROFILE_IMAGE).scaled(90, 90, QtCore.Qt.KeepAspectRatio, QtCore.Qt.SmoothTransformation)) if os.path.exists(PROFILE_IMAGE) else profile.setText("")
        profile.setAlignment(QtCore.Qt.AlignCenter)
        sidebar_layout.addWidget(profile)

        greeting = QtWidgets.QLabel(f"✨ Good {self.get_greeting()}!")
        greeting.setAlignment(QtCore.Qt.AlignCenter)
        greeting.setStyleSheet("color: white; font-size: 18px; font-weight: bold;")
        sidebar_layout.addWidget(greeting)

        buttons_info = [
            ("Add Reminder", "icons/calendar-plus.png", self.add_reminder),
            ("Today’s Reminders", "icons/calendar.png", self.view_reminders),
            ("Emergency Contacts", "icons/phone.png", self.show_contacts),
            ("Add Contact", "icons/user-plus.png", self.add_contact),
            ("Write Note", "icons/edit.png", self.add_note),
            ("My Notes", "icons/book.png", self.view_notes),
            ("Talk to AI", "icons/chat.png", self.talk_to_carecall),
            ("Health Tip", "icons/heart.png", self.get_health_tip),
            ("AI News", "icons/web.png", self.open_health_web),
            ("Exit", "icons/exit.png", self.close)
        ]

        for text, icon, action in buttons_info:
            btn = SidebarButton(text, icon)
            btn.clicked.connect(action)
            sidebar_layout.addWidget(btn)

        sidebar_layout.addStretch()

        # Content Area
        self.content = QtWidgets.QTextBrowser()
        self.content.setHtml(f"<div style='text-align:center; font-size:24px; color:{text_color};'>👋 Welcome to <b>CareCall</b><br><small>How can I help you today?</small></div>")
        self.content.setStyleSheet(f"""
            QTextBrowser {{
                background: transparent;
                border: none;
                padding: 50px;
                font-family: 'Segoe UI';
                font-size: 18px;
                color: {text_color};
            }}
        """)

        main_layout.addWidget(self.sidebar)
        main_layout.addWidget(self.content, 1)

    def get_greeting(self):
        hour = datetime.datetime.now().hour
        if hour < 12:
            return "Morning"
        elif hour < 17:
            return "Afternoon"
        else:
            return "Evening"

    def show_message(self, title, message):
        QtWidgets.QMessageBox.information(self, title, message)

    def add_reminder(self):
        dialog = QtWidgets.QDialog(self)
        dialog.setWindowTitle("Add Reminder")
        layout = QtWidgets.QFormLayout(dialog)
        task_input = QtWidgets.QLineEdit()
        calendar = QtWidgets.QCalendarWidget()
        layout.addRow("Task:", task_input)
        layout.addRow("Date:", calendar)
        btn = QtWidgets.QPushButton("Add")
        layout.addWidget(btn)
        btn.clicked.connect(lambda: self.finish_add_reminder(dialog, task_input, calendar))
        dialog.exec_()

    def finish_add_reminder(self, dialog, task_input, calendar):
        task = task_input.text()
        date = calendar.selectedDate().toString("yyyy-MM-dd")
        if task:
            self.reminders.append({"task": task, "date": date})
            save_data(REMINDERS_FILE, self.reminders)
            self.show_message("Success", "Reminder added!")
            dialog.accept()

    def view_reminders(self):
        todays = get_today_reminders(self.reminders)
        if not todays:
            self.content.setHtml("<h3>No reminders for today.</h3>")
        else:
            html = "<h3>Today’s Reminders:</h3><ul>"
            for r in todays:
                html += f"<li>{r['task']}</li>"
            html += "</ul>"
            self.content.setHtml(html)

    def show_contacts(self):
        html = "<h3>Emergency Contacts:</h3><ul>"
        for c in self.contacts:
            html += f"<li>{c['name']}: {c['phone']}</li>"
        html += "</ul>"
        self.content.setHtml(html)

    def add_contact(self):
        name, ok = QtWidgets.QInputDialog.getText(self, "Add Contact", "Contact Name:")
        if ok and name:
            phone, ok = QtWidgets.QInputDialog.getText(self, "Add Contact", "Phone Number:")
            if ok and phone:
                self.contacts.append({"name": name, "phone": phone})
                save_data(CONTACTS_FILE, self.contacts)
                self.show_message("Success", "Contact added!")

    def talk_to_carecall(self):
        chat = AIChatDialog(self)
        chat.exec_()

    def add_note(self):
        note, ok = QtWidgets.QInputDialog.getMultiLineText(self, "Add Note", "Write your note:")
        if ok and note:
            self.notes.append({"note": note, "timestamp": datetime.datetime.now().isoformat()})
            save_data(NOTES_FILE, self.notes)
            self.show_message("Saved", "Your note was saved.")

    def view_notes(self):
        html = "<h3>Recent Notes:</h3><ul>"
        for n in self.notes[-10:]:
            html += f"<li>{n['note']} <small>({n['timestamp'].split('T')[0]})</small></li>"
        html += "</ul>"
        self.content.setHtml(html)

    def get_health_tip(self):
        tips = [
            "Drink more water!",
            "Stretch every morning!",
            "Talk to a loved one today.",
            "Fresh air lifts your mood.",
            "Limit screen time and rest."
        ]
        tip = random.choice(tips)
        engine.say(tip)
        engine.runAndWait()
        self.content.setHtml(f"<h3>Health Tip:</h3><p>{tip}</p>")

    def open_health_web(self):
        webbrowser.open("https://openai.com/research")

    def init_demo_data(self):
        if not self.contacts:
            self.contacts.extend([
                {"name": "Son", "phone": "9876543210"},
                {"name": "Neighbor", "phone": "1234567890"}
            ])
            save_data(CONTACTS_FILE, self.contacts)
        if not self.reminders:
            demo_reminders = [
                {"task": "Take medicine", "date": datetime.date.today().isoformat()},
                {"task": "Walk for 15 min", "date": datetime.date.today().isoformat()}
            ]
            self.reminders.extend(demo_reminders)
            save_data(REMINDERS_FILE, self.reminders)

def main():
    app = QtWidgets.QApplication(sys.argv)
    app.setStyle("Fusion")
    window = CareCallApp()
    window.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0